In [11]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F

from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler

from torch.utils.data import Dataset, TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc

In [12]:
X, y = make_moons(n_samples=100, noise=0.3, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

In [17]:
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_val = sc.transform(X_val)

In [29]:
x_train_tensor = torch.as_tensor(X_train).float()
y_train_tensor = torch.as_tensor(y_train.reshape(-1, 1)).float()
x_val_tensor = torch.as_tensor(X_val).float()
y_val_tensor = torch.as_tensor(y_val.reshape(-1, 1)).float()
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=16)

In [33]:
torch.manual_seed(42)
model = nn.Sequential()
model.add_module('logistic', nn.Linear(2, 1))
model.add_module('sigmoid', nn.Sigmoid())
print(model.state_dict())

OrderedDict([('logistic.weight', tensor([[0.5406, 0.5869]])), ('logistic.bias', tensor([-0.1657]))])


In [53]:
dummy_labels = torch.tensor([1.0, 0.0])
dummy_predictions = torch.tensor([0.8, 0.2])
positive_pred = dummy_predictions[dummy_labels ==1]
first_summation = torch.log(positive_pred).sum()
negative_pred = dummy_predictions[dummy_labels ==0]
second_summation = torch.log(1-negative_pred).sum()
n_total = dummy_labels.size(0)
loss = -(first_summation + second_summation) / n_total
loss

tensor(0.2231)

In [52]:
summation = torch.sum(dummy_labels * torch.log(dummy_predictions) + (1-dummy_labels) * torch.log(1 - dummy_predictions))
loss = - summation / n_total
loss

tensor(0.2231)